In [151]:
import numpy as np
import json
import inflect
import re
#https://www.instructables.com/How-Ingredients-Behave-In-A-Cookie-Recipe/

In [152]:
#import data
with open('recipes_raw_nosource_ar.json') as json_file:
    data = json.load(json_file)

In [153]:
#get the cookies
new_data = []
for value in list(data.values()):
    if 'title' in value:
        if 'Cookie' in value['title'] or 'cookie' in value['title']:
            new_data.append(value)

In [154]:
# filtering and stuff
# remove non ingrediets
# remove advertisement at the end of ingredients
for elem in new_data:
    for item in elem['ingredients'][:]:
        if not item[0].isdigit():
            elem['ingredients'].remove(item)
    for i, s in enumerate(elem['ingredients']):
        elem['ingredients'][i] =  elem['ingredients'][i][:len(elem['ingredients'][i]) - 14]

In [155]:
#remove () and things after ,
for elem in new_data:
    for i, s in enumerate(elem['ingredients']):
        elem['ingredients'][i] = re.sub('\(.*?\)', '', elem['ingredients'][i])
        elem['ingredients'][i] = elem['ingredients'][i].split(",", 1)[0]

In [156]:

SEPARATOR_RE = re.compile(r'^([\d\s*[\d\.,/]*)\s*(.+)')


def normalize(st):
    """

    :param st:
    :return:
    """
    return re.sub(r'\s+', ' ', SEPARATOR_RE.sub('\g<1> \g<2>', st)).strip()


def escape_re_string(text):
    """

    :param text:
    :return:
    """
    text = text.replace('.', '\.')
    return re.sub(r'\s+', ' ', text)

In [157]:


import re
from itertools import chain


UNITS = {"cup": ["cups", "cup", "c.", "c"], "fluid_ounce": ["fl. oz.", "fl oz", "fluid ounce", "fluid ounces"],
         "package": ["package","packages"],"jar":["jar","jars"], "box":["box","boxes"],
         "gallon": ["gal", "gal.", "gallon", "gallons"], "ounce": ["oz", "oz.", "ounce", "ounces"],
         "pint": ["pt", "pt.", "pint", "pints"], "pound": ["lb", "lb.", "pound", "pounds"],
         "quart": ["qt", "qt.", "qts", "qts.", "quart", "quarts"],
         "tablespoon": ["tbsp.", "tbsp", "T", "T.", "tablespoon", "tablespoons", "tbs.", "tbs"],
         "teaspoon": ["tsp.", "tsp", "t", "t.", "teaspoon", "teaspoons"],
         "gram": ["g", "g.", "gr", "gr.", "gram", "grams"], "kilogram": ["kg", "kg.", "kilogram", "kilograms"],
         "liter": ["l", "l.", "liter", "liters"], "milligram": ["mg", "mg.", "milligram", "milligrams"],
         "milliliter": ["ml", "ml.", "milliliter", "milliliters"], "pinch": ["pinch", "pinches"],
         "dash": ["dash", "dashes"], "touch": ["touch", "touches"], "handful": ["handful", "handfuls"],
         "stick": ["stick", "sticks"], "clove": ["cloves", "clove"], "can": ["cans", "can"], "large": ["large"],
         "small": ["small"], "scoop": ["scoop", "scoops"], "filets": ["filet", "filets"], "sprig": ["sprigs", "sprig"]}

NUMBERS = ['seventeen', 'eighteen', 'thirteen', 'nineteen', 'fourteen', 'sixteen', 'fifteen', 'seventy', 'twelve',
           'eleven', 'eighty', 'thirty', 'ninety', 'twenty', 'seven', 'fifty', 'sixty', 'forty', 'three', 'eight',
           'four', 'zero', 'five', 'nine', 'ten', 'one', 'six', 'two', 'an', 'a']

prepositions = ["of"]

a = list(chain.from_iterable(UNITS.values()))
a.sort(key=lambda x: len(x), reverse=True)
a = map(escape_re_string, a)

PARSER_RE = re.compile(
    r'(?P<quantity>(?:[\d\.,][\d\.,\s/]*)?\s*(?:(?:%s)\s*)*)?(\s*(?P<unit>%s)\s+)?(\s*(?:%s)\s+)?(\s*(?P<name>.+))?' % (
        '|'.join(NUMBERS), '|'.join(a), '|'.join(prepositions)))


def parse(st):
    """

    :param st:
    :return:
    """
    st = normalize(st)
    res = PARSER_RE.match(st)

    return {
        'measure': ((res.group('quantity') or '').strip() + ' ' + (res.group('unit') or '').strip()).lstrip().rstrip(),
        'name': (res.group('name') or '').strip()
    }


In [158]:
#from ingredient_parser.en import parse 
for elem in new_data:
    for i, s in enumerate(elem['ingredients']):
        elem['ingredients'][i] =  parse(elem['ingredients'][i])

#print(new_data)

In [159]:
#singularize
p = inflect.engine()
for elem in new_data:
    for i, s in enumerate(elem['ingredients']):
        #if (elem['ingredients'][i]['name'] == "egg yolks"):
        #    print( p.singular_noun(elem['ingredients'][i]['name']))
        if p.singular_noun(elem['ingredients'][i]['name']) != False:
            elem['ingredients'][i]['name'] = p.singular_noun(elem['ingredients'][i]['name'])

In [160]:
#new_data

In [161]:
with open('roles.json') as json_file:
    roles = json.load(json_file)

for elem in new_data:
    for i, s in enumerate(elem['ingredients']):
        for role,role_values in roles.items():
            for ing_role in role_values:
                #print(ing_role)
                if ing_role in elem['ingredients'][i]['name'].lower() and 'role' not in elem['ingredients'][i]:
                    #print(ing_role,elem['ingredients'][i]['name'])
                    elem['ingredients'][i]['role'] = role
        

In [162]:
#remove non-rolled ingredients
for elem in new_data:
    for ingre in elem['ingredients'][:]:        
        if 'role' not in ingre:
            elem['ingredients'].remove(ingre)

In [163]:
#remove fractions
from fractions import Fraction
for elem in new_data:
    for i, s in enumerate(elem['ingredients']):
        words = elem['ingredients'][i]['measure'].split()
        total = 0
        if '/' in words[0]:
            total = float(Fraction(words[0]))
        else:
            total = float(words[0])
        if len(words)> 1 and '/' in words[1]:
            total += float(Fraction(words[1]))
            total = round(total,3)
            arr = [str(total)] + words[2:]
            #print(" ".join(arr))
            elem['ingredients'][i]['measure'] = " ".join(arr)
            #print(arr)
            #print([total].extend(words[1:]))
        elif total > 0:
            total = round(total,3)
            arr = [str(total)] + words[1:]
            #print(" ".join(arr))
            elem['ingredients'][i]['measure'] = " ".join(arr)
            #print(arr)
            #print(total)

In [164]:
for elem in new_data:
    for i, s in enumerate(elem['ingredients']):
        #elem['ingredients'][i] =  parse(elem['ingredients'][i])
        words = elem['ingredients'][i]['measure'].split()
        elem['ingredients'][i]['quantity'] = words[0]
        if len(words) > 1:
            elem['ingredients'][i]['measure'] = words[1]
        else:
            elem['ingredients'][i]['measure'] = 'units'
        

In [165]:
cup_equivalent = {
    'cup':1.0,
    'cups':1.0,
    'package':3.0,#random
    'packages': 3.0,
    'box': 4.0,#random
    'boxes': 4.0,
    'tablespoon':0.0625,
    'tablespoons':0.0625,
    'teaspoon': 0.02,
    'teaspoons': 0.02,
    'pound':3.5,
    'pounds':3.5,
    'pint':2,
    'pints':2,
}
def quantity_norm(ingredient):
    if ingredient['measure'] not in cup_equivalent:
        print(ingredient)
    return float(ingredient['quantity']) * cup_equivalent[ingredient['measure']]

In [166]:
#normalize weights
flour_value = 5.0
for recipe in new_data:
    sum = 0
    for ingredient in recipe['ingredients']:
        if ingredient['role'] == 'flour':
            sum += quantity_norm(ingredient)
    if sum == 0:
        sum = flour_value
    for i, s in enumerate(recipe['ingredients']):
        recipe['ingredients'][i]['quantity'] = round(float(recipe['ingredients'][i]['quantity']) * flour_value/sum,1)


In [167]:
jsonString = json.dumps(new_data, indent=2, default=str)
jsonFile = open("data.json", "w")
jsonFile.write(jsonString)
jsonFile.close()